In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append('../Customized_Components') #this path is not right for this PC. Maybne this will work?
from bandaged_dolan import DolanJunctionBandage
from houcklab_qubit import DiffTransmonRounded

import warnings

import shapely
from CoupledLineTee import CoupledLineTee
from dolan_junction import DolanJunction
from LaunchpadWirebondCustom import LaunchpadWirebondCustom
from shapely.errors import ShapelyDeprecationWarning
from single_pad_transmon_pocket import TransmonPocket_Single

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
from collections import OrderedDict

import Default_res_params as dp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import os

# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
# import pyEPR as epr
from qiskit_metal import Dict, Headings, MetalGUI, designs, draw
# from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
# from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
# from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
# from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
# from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
# from qiskit_metal.toolbox_metal.parsing import *


In [2]:
gui = MetalGUI(dp.design)
design = dp.design
design.overwrite_enabled = True
c1, renderer_q3d, q3d = dp.init_q3d_sim()
eig_all, hfss_renderer, hfss = dp.init_hfss_sim()

## Calculate Qubit Params

In [3]:
import Transmon_property as tp

In [4]:
import scqubits as scq
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

from scipy.optimize import root_scalar

In [5]:
ratio = 65
Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

def inductance_to_cap(L,ratio = ratio):
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    return (c.e.si**2 / (2 * EC)/c.h).to(u.fF)

def target_function(L, desired_frequency, ratio):
    L = L * u.nH  # Convert L to nanohenrys
    calculated_frequency = calculate_qubit_freq(L, ratio)
    return calculated_frequency - desired_frequency
def calculate_qubit_freq(L,ratio = ratio):
    
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    ng = 0.0   # Offset charge
    nlev = 10  # Number of energy levels to consider

    # Create a Transmon object
    transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

    # Calculate the eigenenergies
    eigenenergies = transmon.eigenvals()

    # Calculate the transition frequencies (in GHz)
    transition_frequencies = np.diff(eigenenergies)

    # Calculate the anharmonicity (in GHz)
    anharmonicity = transition_frequencies[1] - transition_frequencies[0]
    return transition_frequencies[0]
    
class TransmonQubit():
    def __init__(self, freq, ratio):
        self.freq = freq
        self.ratio = ratio
        self.C = None,
        self.L = None
        self.EJ = None
        self.EC = None
        self.update_params()
        self.alpha = self.EC
        
    def update_params(self):
        self.C, self.L = self.qubit_freq_to_cap()
        Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

        # Calculate the Josephson energy EJ and charging energy EC
        EJ = (((Phi0/2/np.pi)**2/self.L).to(u.J)/c.h).to(u.GHz)#, equivalencies=u.spectral())  # Convert to GHz
        EC = (c.e.si**2 / (2 * self.C)/c.h).to(u.GHz)
        self.EJ = EJ
        self.EC = EC
        
    def get_params(self):
        return [self.freq, self.ratio, self.qubit_capacitance, self.qubit_inductance, self.EJ, self.EC]
    def get_qcap(self):
        return self.C
    
    def get_qind(self):
        return self.L
    
    def get_EJ(self):
        return self.EJ
    
    def get_EC(self):
        return self.EC
    
    def qubit_freq_to_cap(self, ):
        desired_frequency = self.freq   # Desired frequency in GHz
        ratio = self.ratio
        # Use scipy.optimize.root_scalar to find the root
        def target_function(L, desired_frequency,):
            self.L = L * u.nH  # Convert L to nanohenrys
            self.calculate_qubit_freq()
            calculated_frequency = self.freq.to(u.GHz).value
            return calculated_frequency - desired_frequency
        result = root_scalar(target_function, args=(desired_frequency.to(u.GHz).value,), bracket=[1, 100], method='brentq')

        # Print the result
        if result.converged:
            L_optimal = result.root
            print(f"Optimal inductance L: {L_optimal} nH")
        else:
            print("Root finding did not converge")
        c_optimal = inductance_to_cap(L_optimal*u.nH, ratio = ratio)
        return c_optimal, L_optimal*u.nH
    def calculate_qubit_freq(self,):
        L = self.L
        ratio = self.ratio
        EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
        #, equivalencies=u.spectral())  # Convert to GHz
        EC = EJ/ratio
        ng = 0.0   # Offset charge
        nlev = 10  # Number of energy levels to consider

        # Create a Transmon object
        transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

        # Calculate the eigenenergies
        eigenenergies = transmon.eigenvals()

        # Calculate the transition frequencies (in GHz)
        transition_frequencies = np.diff(eigenenergies)

        # Calculate the anharmonicity (in GHz)
        anharmonicity = transition_frequencies[1] - transition_frequencies[0]
        self.freq = transition_frequencies[0]*u.GHz
        self.alpha = anharmonicity*u.GHz
    

Design specifics

In [6]:
from Transmon_specifications import find_junction_area, find_junction_capacitance

In [7]:
def pad_len_from_C(c):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*c.to(u.fF).value + intercept 
def unitless_find_j_area(L):
    return find_junction_area(L*u.nH,Jc = 0.42*u.uA/u.um**2).to(u.um**2).value

def unitless_find_pad_len(C):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*C + intercept

In [8]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,-1,0],
              [0,1,0,0],
              [1,0,1,0],
              [0,0,0,1]])
    cap_matrix[0,2] -= cj
    cap_matrix[2,0] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[0,0]
    coupling_capacitance = np.reciprocal(inverse)[0,1]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    g = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    return [Ec,g,qubit_capacitance,coupling_capacitance]
    

In [9]:
get_cap_params(np.array([[80,0,0,0],[0,80,0,0],[0,0,80,0],[0,0,0,80]]),0.5)

[<Quantity 0.48425573 GHz>, <Quantity 0. MHz>, 40.0, inf]

## Resonator calculations

In [10]:
import astropy.units as u
import astropy.constants as c
import scipy.optimize as optimize

# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 100*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width
Z0 = 50*u.Ohm #characteristic impedance
import qiskit_metal.analyses as analyses

In [11]:
freq = 6*u.GHz.si
line_width = 15*u.um
line_gap = 8.30*u.um

Optimal pin-gap ratio for resonator and feedlines

In [12]:
class ReadoutResonator():
    def __init__(self, freq, short_on_one_end = False,optimal_pin = None, optimal_gap = None ):
        self.freq = freq
        self.short_on_one_end = short_on_one_end
        self.substrate_thickness = 350*u.um
        self.film_thickness = 100*u.nm
        if (optimal_gap is None) and (optimal_pin is None):
            self.pin = 15*u.um
            self.optimize_gap()
        elif optimal_gap is None:
            self.pin = optimal_pin
            self.optimize_gap()
        elif optimal_pin is None:
            self.gap = optimal_gap
            self.optimize_pin()
        
        self.param_from_res_freq(freq, short_on_one_end = short_on_one_end)
        self.C = np.pi/2/self.freq/2/np.pi/self.Z0
        
    def get_params(self):
        return [self.L, self.C, self.target_length]
    
    def get_L(self):
        return self.L
    
    def get_C(self):
        return self.C
    
    def get_target_length(self):
        return self.target_length
    
    def param_from_res_freq(self,freq, short_on_one_end = False):
        self.Lk, self.Lext,self.Cl,self.G,self.Z0,self.etf,self.Cstar = analyses.cpw_calculations.lumped_cpw(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)
        self.Lk*=u.H
        self.Lext*=u.H
        self.Cl*=u.F/u.m
        
        self.G*=u.S
        self.Z0*=u.Ohm
        self.etf*=u.H/u.m
        self.Cstar*=u.F
        target_length = analyses.cpw_calculations.guided_wavelength(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)[0]*u.m
        if short_on_one_end:
            target_length /= 4
        else:
            target_length /= 2
        self.len = target_length
    def optimize_gap(self):
        def f(line_gap):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            self.pin.to(u.m).value,
                                            line_gap,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_gap = optimize.fsolve(f,self.pin.si.value)[0]*u.m
        self.gap = optimized_gap.to(u.um)
    def optimize_pin(self):
        def f(line_width):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            line_width,
                                            self.gap.si.value,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_pin = optimize.fsolve(f,self.gap.si.value)[0]*u.m
        self.pin = optimized_pin.to(u.um)

### dispersive readout function

In [13]:
class DispersiveReadout():
    def __init__(self, qubit, res, T1, c_qr = None, c_tr = None):
        self.qubit = qubit
        self.res = res
        self.pT1 = T1
        self.optimize_cqr()
        self.optimize_ctr()
        
    def get_params(self):
        return [self.qubit, self.res, self.c_qr, self.c_tr]
    
    def get_cqr(self):
        return self.c_qr
    
    def get_ctr(self):
        return self.c_tr
    
    def get_qubit(self):
        return self.qubit
    
    def get_res(self):
        return self.res
    def calc_g(self):
        c_qr = self.c_qr
        Cq = self.qubit.C
        Cr = self.res.C
        wq = self.qubit.freq*np.pi*2
        wr = self.res.freq*np.pi*2
        EJ = self.qubit.EJ
        EC = self.qubit.EC
        return c_qr/Cq*np.sqrt(c.e.si**2*wr/c.hbar/Cr)*(EJ/8/EC)**0.25
    def update_g(self):
        self.g = self.calc_g()
    def calc_kappa(self):
        res = self.res
        c_tr = self.c_tr
        return (analyses.em.kappa_calculation.kappa_in(res.freq.si.value,
                                       c_tr.si.value,
                                       res.freq.si.value)*u.Hz).to(u.MHz)
    def update_kappa(self):
        self.kappa = self.calc_kappa()
    def calc_chi(self):
        qubit = self.qubit
        resonator = self.res
        delta = (qubit.freq-resonator.freq)*2*np.pi
        sum_freq = (qubit.freq+resonator.freq)*2*np.pi
        alpha = qubit.EC
        return 2*self.g**2*(alpha/delta/(delta+alpha)+alpha/sum_freq/(sum_freq+alpha))
    def update_chi(self):
        self.chi = self.calc_chi()
    def calc_purcellT1(self,):
        qubit = self.qubit
        res = self.res
        
        delta = (qubit.freq-res.freq)*2*np.pi
        
        purcellGamma = self.g**2*self.kappa/delta**2
        
        return 1/purcellGamma/2/np.pi
    def calc_purcellT1q(self,update_g = True, update_chi = True):
        '''taking kappa = 2*chi for the optimal SNR, can directly get the T1q'''
        c_qr = self.c_qr
        qubit = self.qubit
        res = self.res
        if update_g:
            self.update_g()
        if update_chi:
            self.update_chi()
        delta = (qubit.freq-res.freq)*2*np.pi
        
        kappa_val = self.chi*2
        
        purcellGamma = self.g**2*kappa_val/delta**2
        return 1/purcellGamma/2/np.pi
    def optimize_cqr(self,):
        '''find the optimal coupling capacitance'''
        target_t1 = self.pT1
        c_qr = 1*u.fF
        self.c_qr = c_qr
        while self.calc_purcellT1q().to(u.ms) > 1.01*target_t1:
            c_qr += .1*u.fF
            self.c_qr = c_qr
        return c_qr
    def optimize_ctr(self, target_kappa=None):
        '''find the optimal coupling capacitance'''
        if target_kappa is None:
            target_kappa = self.chi*2
        c_tr = 1*u.fF
        self.c_tr = c_tr
        self.update_kappa()
        while self.kappa < target_kappa:
            c_tr += .1*u.fF
            self.c_tr = c_tr
            self.update_kappa()
        # return c_tr


In [14]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 13.676453476080171 nH


In [15]:
default_options = Dict(
        cut_l='800um',
        cut_h='1200um',
        gap='20um',
        w = '90um',
        l = '900um',
        r = '60um',
        cpw_l = '50um',
        coupling_gap = '40um',
        JJ_cutout_w = '70um',
        JJ_cutout_h = '70um',
        JJ_cutout_r = '150um',
        JJ_c_contact_l = '40um',
        JJ_c_contact_r = '2.5um',
        JJ_c_contact_w = '10um',
        coupling_d = '100um',
        coupling_pad_w = '90um',
        coupling_r = '40um',
        cpw_pin = '10um',
        chip='main',
        resolution = '10',
        junction = 'False',
        orientation = '0',
        JJ_c_contact_shortl = '10um',
        istunnel = 'False')

## Leonding method

In [16]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params_dim4(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,0,0],
                  [0,1,1,0],
                  [0,1,-1,0],
                  [0,0,0,1]])
    cap_matrix[1,2] -= cj
    cap_matrix[2,1] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[1,1] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[2,2]
    coupling_capacitance = np.reciprocal(inverse)[2,3]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    J = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    # gp = (1/c.hbar*c.e.si*np.sqrt(np.pi*c.hbar/2/z0)*(32*Ec/Ej)**0.25).to(u.MHz)*coupling_capacitance
    # print(gp)
    return [Ec,J,qubit_capacitance,coupling_capacitance]
def get_cap_params_dim2(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,1,0],
                  [1,-1,0],
                  [0,0,1]])
    cap_matrix[0,1] -= cj
    cap_matrix[1,0] -= cj
    cap_matrix[1,1] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[1,1]
    coupling_capacitance = np.reciprocal(inverse)[1,2]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    J = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    
    # gp = (1/c.hbar*c.e.si*np.sqrt(np.pi*c.hbar/2/z0)*(32*Ec/Ej)**0.25).to(u.MHz)*coupling_capacitance
    # print(gp)
    return [Ec,J,qubit_capacitance,coupling_capacitance]
def qubit_zpf(E_J, E_c):
    z_Q = (1/2*np.pi)*np.sqrt(8*E_c/E_J)
    return np.sqrt(1/(4*np.pi*z_Q))
def res_zpf(Z_0 = 50*u.Ohm):
    R_Q = c.h/(2*c.e.si**2)
    Z = 2*Z_0/np.pi
    z= Z/R_Q
    return np.sqrt(1/(4*np.pi*z))
    

In [17]:
phi0 = c.h/2/c.e.si
epsilon_r = 11.45
z0 = 50*u.Ohm

def g_leonDing_old(dim = 4, c1 = c1, cj = 2, Lj = 13*u.nH):
    df = c1.sim.capacitance_matrix
    df1 = copy.deepcopy(df)
    df1['ground_main_plane']['ground_main_plane'] += res.C.to(u.fF).value
    df1['resonator_pad_Q1']['resonator_pad_Q1'] += res.C.to(u.fF).value
    df1['resonator_pad_Q1']['ground_main_plane'] -= res.C.to(u.fF).value
    df1['ground_main_plane']['resonator_pad_Q1'] -= res.C.to(u.fF).value
    cap = df1.to_numpy()
    if dim == 3:
        capp = cap[1:,1:]
        Ec, J,_,_ = get_cap_params(capp,cj)
    elif dim == 4:
        capp = cap
        Ec, J,_,_ = get_cap_params_dim4(capp,cj) 
        
    
    # Ec = (c.e.si**2/2/Cq)/c.h
    Ej = ((phi0/2/np.pi)**2/Lj)/c.h   
    g = (qubit_zpf(Ej,Ec)*J*res_zpf()).to(u.MHz)
    
    
    
    return g, Ec, Ej, 

In [18]:
def g_leonDing(dim = 4, c1 = c1, cj = 2, Lj = 13*u.nH):
    df = c1.sim.capacitance_matrix
    df1 = copy.deepcopy(df)
    
    
    # df1['ground_main_plane']['ground_main_plane'] += res.C.to(u.fF).value
    # df1['resonator_pad_Q1']['resonator_pad_Q1'] += res.C.to(u.fF).value
    # df1['resonator_pad_Q1']['ground_main_plane'] -= res.C.to(u.fF).value
    # df1['ground_main_plane']['resonator_pad_Q1'] -= res.C.to(u.fF).value
    cap = df1.to_numpy()
    cap[1,1] += cap[0,1]
    cap[2,2] += cap[0,2]
    cap[3,3] += cap[0,3]
    
    cap[0,3] += res.C.to(u.fF).value
    cap[3,0] += res.C.to(u.fF).value
    cap[0,0] -= res.C.to(u.fF).value
    cap[3,3] -= res.C.to(u.fF).value
    
    
    if dim == 3:
        capp = cap[1:,1:]
        Ec, J,_,_ = get_cap_params(capp,cj)
    elif dim == 4:
        capp = cap
        Ec, J,_,_ = get_cap_params_dim4(capp,cj) 
        
    
    # Ec = (c.e.si**2/2/Cq)/c.h
    Ej = ((phi0/2/np.pi)**2/Lj)/c.h   
    g = (qubit_zpf(Ej,Ec)*J*res_zpf()).to(u.MHz)
    
    
    
    return g, Ec, Ej, 

In [19]:
# g_leonDing(dim = 4, c1 = c1, cj = 2, Lj = 13*u.nH)

In [20]:
qubit.alpha = 0.2*u.GHz
dr.calc_g().to(u.MHz)

<Quantity 180.82445195 MHz>

In [21]:
c1.setup.junctions = Dict({'Lj': qubit.L.to(u.nH).value, 'Cj': 2})
c1.setup.freq_readout = 6.0
c1.setup.freq_bus = []

c1.run_lom()
c1.lumped_oscillator_all

09:34PM 54s WARNING [run_lom]: Please initialize the capacitance_matrix before executing this method.`self.sim.capacitance_matrix = pd.DataFrame(...)`


{}

## Alex Page Thesis method

In [22]:
df = c1.sim.capacitance_matrix

In [23]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45
z0 = 50*u.Ohm


Cq = 20*u.fF
Lj = 13*u.nH
Ec = (c.e.si**2/2/Cq)/c.h


In [24]:
def g_alexPage(c1 = c1,wr = 6*u.GHz, Lj = 13*u.nH):
    df = c1.sim.capacitance_matrix
    c_1 = -df['pad_left_Q1']['resonator_pad_Q1']*u.fF
    c_2 = -df['pad_right_Q1']['ground_main_plane']*u.fF
    c_3 = -df['pad_right_Q1']['resonator_pad_Q1']*u.fF
    c_4 = -df['pad_left_Q1']['ground_main_plane']*u.fF
    ct = -df['pad_left_Q1']['pad_right_Q1']*u.fF

    c_phi = (c_2+c_3)*(c_1+c_4)/(c_1+c_2+c_3+c_4)+2*u.fF+ct
    c_ceff = (c_1**2-c_3*c_4)/(c_1+c_2+c_3+c_4+ct)
    beta = (c_1*c_2-c_3*c_4)/((c_2+c_3)*(c_1+c_4)+ct*(c_1+c_2+c_3+c_4))
    Ec = (c.e.si**2/2/c_phi)/c.h
    Ej = ((phi0/2/np.pi)**2/Lj)/c.h
    w01 = np.sqrt(8*Ec*Ej)
    delta = wr-w01
    c_res = np.pi/2/wr/z0
    Vrms = np.sqrt(c.hbar*wr/2/c_res)
    g = c.e.si*Vrms*beta/c.hbar*np.sqrt(2)*(Ej/8/Ec)**0.25
    return g.to(u.MHz), Ec.to(u.MHz), Ej.to(u.GHz), w01.to(u.GHz)

In [25]:
# g_alexPage()

## Qiskit Metal

In [210]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

In [26]:
from datetime import datetime

def get_current_time_string():
    now = datetime.now()
    return now.strftime("%Y-%m-%d%H-%M")

# Example usage
print(get_current_time_string())

2024-11-1721-34


In [27]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)

Optimal inductance L: 13.676453476080171 nH


In [28]:
default_options = Dict(
        cut_l='800um',
        cut_h='1200um',
        gap='20um',
        w = '90um',
        l = '900um',
        r = '60um',
        cpw_l = '50um',
        coupling_gap = '40um',
        JJ_cutout_w = '70um',
        JJ_cutout_h = '70um',
        JJ_cutout_r = '150um',
        JJ_c_contact_l = '40um',
        JJ_c_contact_r = '2.5um',
        JJ_c_contact_w = '10um',
        coupling_d = '100um',
        coupling_pad_w = '90um',
        coupling_r = '40um',
        cpw_pin = '10um',
        chip='main',
        resolution = '10',
        junction = 'False',
        orientation = '0',
        JJ_c_contact_shortl = '10um',
        istunnel = 'False')

### Define qubit, resonator params

In [29]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6.5*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)


Optimal inductance L: 13.676453476080171 nH


In [30]:
q1 = DiffTransmonRounded(design,'Q1',options = default_options)
gui.rebuild()

False
junction added
False
junction added


In [31]:
raise Exception('stop!')

Exception: stop!

### Simulation

In [ ]:
raise Exception('stop!')

Exception: stop!

In [ ]:
qubit = TransmonQubit(4.8*u.GHz, ratio = 65)
res = ReadoutResonator(6.5*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 11.397044563400145 nH


In [ ]:
default_options_q2

{'cut_l': '650um+500um+15um+25um',
 'cut_h': '1200um',
 'gap': '20um',
 'w': '90um',
 'l': '900um',
 'r': '60um',
 'cpw_l': '50um',
 'coupling_gap': '15um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '400um',
 'coupling_pad_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': '10um',
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'coupling_gd': '650um'}

In [ ]:
default_options['coupling_d']+'+500um+'+default_options['coupling_gap']+'+'+default_options['gap']


'250um+500um+15um+35um'

In [ ]:
# default_options['coupling_d'] = '300um'
# default_options['coupling_gap'] = '15um'
# default_options['gap'] = '25um'
# default_options['l'] = '00um'
# default_options['cut_l'] = '900um'
default_options = default_options_q3
c1, renderer_q3d, q3d = dp.init_q3d_sim()
# sweep_param = 'coupling_d'
sweep_param = 'gap'
sweep_array = np.arange(5,40,10)
date = get_current_time_string()
dp.change_inductance([f'{(qubit.L.to(u.nH).value):.2f}nH'],eig_all=eig_all,c1 = c1)

for i,l in enumerate(sweep_array):
    default_options[sweep_param] = f'{l}um'
    default_options['cut_l'] = default_options['coupling_d']+'+500um+'+default_options['coupling_gap']+'+'+default_options['gap']
    q1 = DiffTransmonRounded(design,'Q1',options = default_options)
    gui.rebuild()
    
    c1.sim.run(components = ['Q1'],open_terminations = [('Q1','cpw_stub')])
    print(l)
    print(c1.sim.capacitance_matrix)
    if i ==0:
        dir = 'data/'+date+f'{sweep_param}_capmat'
        os.mkdir(dir)
    cap_mat = c1.sim.capacitance_matrix
    cap_mat.to_csv(dir+f'/{l}.csv')
    
    
    
    # c1.setup.junctions = Dict({'Lj': qubit.L.to(u.nH).value, 'Cj': 2})
    c1.setup.freq_readout = res.freq.to(u.GHz).value
    c1.setup.freq_bus = []

    c1.run_lom()
    result = pd.DataFrame(c1.lumped_oscillator)
    last_row = result
    for keys in default_options.keys():
        last_row[keys] = design.parse_value(default_options[keys])
    last_row['Lj [nH]'] = qubit.L.to(u.nH).value
    
    g_alex,Ec_alex, Ej_alex, wr_alex = g_alexPage(c1 = c1, Lj = qubit.L)
    # g_leon, Ec_leon, Ej_leon = g_leonDing(c1 = c1, Lj = qubit.L)
    # g_leon3, Ec_leon3, Ej_leon3 = g_leonDing(c1 = c1,dim = 3, Lj = qubit.L)
    
    last_row['g_alex [MHz]'] = g_alex.to(u.MHz).value
    last_row['Ec_alex [MHz]'] = Ec_alex.to(u.MHz).value
    last_row['Ej_alex [GHz]'] = Ej_alex.to(u.GHz).value
    last_row['wr_alex [GHz]'] = wr_alex.to(u.GHz).value
    # last_row['g_leon [MHz]'] = g_leon.to(u.MHz).value
    # last_row['Ec_leon [MHz]'] = Ec_leon.to(u.MHz).value
    # last_row['Ej_leon [GHz]'] = Ej_leon.to(u.GHz).value
    
    # last_row['g_leon3 [MHz]'] = g_leon3.to(u.MHz).value
    # last_row['Ec_leon3 [MHz]'] = Ec_leon3.to(u.MHz).value
    # last_row['Ej_leon3 [GHz]'] = Ej_leon3.to(u.GHz).value
    
    dr.g = last_row['gbus'][0]*u.MHz
    
    dr.update_chi()
    chi = dr.chi
    last_row['Chi [MHz]'] = chi.to(u.MHz).value
    qiskit_chi = np.abs(last_row['chi_in_MHz'][0])*u.MHz
    purcell_t1 = dr.calc_purcellT1q(update_g=False)
    target_kappa = 2*chi
    
    dr.chi = qiskit_chi
    purcell_t1_qiskit = dr.calc_purcellT1q(update_g=False, update_chi = False)
    target_kappa_qiskit = 2*chi
    
    
    
    last_row['Purcell T1 [us]'] = purcell_t1.to(u.us).value
    last_row['target kappa [MHz]'] = target_kappa.to(u.MHz).value
    
    last_row['Purcell T1 qiskit [us]'] = purcell_t1_qiskit.to(u.us).value
    last_row['target kappa qiskit [MHz]'] = target_kappa_qiskit.to(u.MHz).value
    
    
    
    
    
    if i ==0:
        df = last_row
    else:
        df = df.append(last_row, ignore_index=True,)

    df.to_csv('data/'+date+f'{sweep_param}_sweep.csv')
    
    if i == len(sweep_array)-1:
        plt.cla()
        plt.scatter(df[sweep_param].values, df['g_alex [MHz]'].values, label = 'g_alex', color = 'red', marker = 'x')
        # plt.scatter(df[sweep_param].values, df['g_leon [MHz]'].values/2, label = 'g_leon', color = 'orange', marker = '.')
        # plt.scatter(df[sweep_param].values, -df['g_leon3 [MHz]'].values, label = 'g_leon3', color = 'green', marker = 'o')
        plt.scatter(df[sweep_param].values, df['gbus'].values, label = 'g_qiskit', color = 'blue', marker = '*')
        plt.xlabel(sweep_param)
        plt.ylabel('g [MHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'g_sweep1.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['Ec_alex [MHz]'].values, label = 'Ec_alex', color = 'red', marker = 'x')
        # plt.scatter(df[sweep_param].values, df['Ec_leon [MHz]'].values, label = 'Ec_leon', color = 'orange', marker = '.')
        plt.scatter(df[sweep_param].values, df['EC'].values, label = 'Ec_qiskit', color = 'blue', marker = 'o')
        # plt.scatter(df[sweep_param].values, df['Ec_leon3 [MHz]'].values, label = 'Ec_leon3', color = 'green', marker = '*')
        plt.xlabel(sweep_param)
        plt.ylabel('Ec [MHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'Ec_sweep.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['Ej_alex [GHz]'].values, label = 'Ec_alex', color = 'red', marker = 'x')
        # plt.scatter(df[sweep_param].values, df['Ej_leon [GHz]'].values, label = 'Ec_leon', color = 'orange', marker = '.')
        plt.scatter(df[sweep_param].values, df['EJ'].values, label = 'Ec_qiskit', color = 'blue', marker = 'o')
        plt.xlabel(sweep_param)
        plt.ylabel('Ej [MHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'Ej_sweep.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['Chi [MHz]'].values, label = 'Chi', color = 'red', marker = 'x')
        plt.scatter(df[sweep_param].values, -df['chi_in_MHz'].values, label = 'Chi_qiskit', color = 'blue', marker = 'o')
        plt.xlabel(sweep_param)
        plt.ylabel('Chi [MHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'Chi_sweep.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['Purcell T1 [us]'].values, label = 'Purcell T1', color = 'red', marker = 'x')
        plt.scatter(df[sweep_param].values, df['Purcell T1 qiskit [us]'].values, label = 'Purcell T1 qiskit', color = 'blue', marker = 'o')
        plt.xlabel(sweep_param)
        plt.ylabel('Purcell T1 [us]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'Purcell_T1_sweep.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['fQ'].values, label = 'qiskit freQ', color = 'red', marker = 'x')
        plt.scatter(df[sweep_param].values, df['wr_alex [GHz]'].values, label = 'alex freq', color = 'blue', marker = 'o')
        plt.xlabel(sweep_param)
        plt.ylabel('fQ [GHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'fQ_sweep.png')
        plt.show()
# q1 = DiffTransmonRounded(design,'Q1',options = default_options)
# gui.rebuild()

INFO 08:12PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]


False
junction added
False
junction added


INFO 08:12PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 08:12PM [analyze]: Analyzing setup Setup
INFO 08:13PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp2k88v428.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 08:13PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp4gcx_bg9.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 08:13PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpckspqjkb.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 08:13PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpr6ap2sof.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 3, False
INFO 08:13PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Loca

5
                   ground_main_plane  pad_left_Q1  pad_right_Q1  \
ground_main_plane          214.64290    -73.33567     -40.57198   
pad_left_Q1                -73.33567    187.94346     -64.62008   
pad_right_Q1               -40.57198    -64.62008     108.23592   
resonator_pad_Q1           -19.65971    -48.03442      -1.88029   

                   resonator_pad_Q1  
ground_main_plane         -19.65971  
pad_left_Q1               -48.03442  
pad_right_Q1               -1.88029  
resonator_pad_Q1           69.99676  
[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.432323 [GHz]
EC 197.511409 [MHz]
EJ 13.616300 [GHz]
alpha -221.269349 [MHz]
dispersion 0.365860 [KHz]
Lq 11.995161 [nH]
Cq 98.071439 [fF]
T1 127.876155 [us]

**Coupling Properties**

tCqbus1 11.054408 [fF]
gbus1_in_MHz 111.581548 [MHz]
χ_bus1 -1.211227 [MHz]
1/T1bus1 1244.602189 [Hz]
T1bus1 127.876155 [us]
Bus-Bus Couplings
False
junction added
False
junction added


INFO 08:13PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 08:13PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 08:13PM [analyze]: Analyzing setup Setup
INFO 08:14PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp7rm9fb90.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 08:14PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpv0rlonv9.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 08:14PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpw3bzv87j.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 08:14PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp9vdka4kp.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, 

...

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.025090 [GHz]
EC 257.479494 [MHz]
EJ 13.616300 [GHz]
alpha -294.308225 [MHz]
dispersion 6.184403 [KHz]
Lq 11.995161 [nH]
Cq 75.230178 [fF]
T1 34.515112 [us]

**Coupling Properties**

tCqbus1 11.496235 [fF]
gbus1_in_MHz 142.457302 [MHz]
χ_bus1 -4.670069 [MHz]
1/T1bus1 4611.166873 [Hz]
T1bus1 34.515112 [us]
Bus-Bus Couplings
False
junction added
False
junction added


INFO 08:15PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 08:15PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 08:15PM [analyze]: Analyzing setup Setup
INFO 08:16PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmppcvq3pt6.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 08:16PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp13dvjwlp.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 08:16PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp40z9t_c_.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 08:16PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpb7t7uewf.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, 

35
                   ground_main_plane  pad_left_Q1  pad_right_Q1  \
ground_main_plane          220.26961    -75.50532     -43.62049   
pad_left_Q1                -75.50532    161.37956     -35.63004   
pad_right_Q1               -43.62049    -35.63004      82.23861   
resonator_pad_Q1           -19.49777    -48.24928      -1.70716   

                   resonator_pad_Q1  
ground_main_plane         -19.49777  
pad_left_Q1               -48.24928  
pad_right_Q1               -1.70716  
resonator_pad_Q1           69.87959  


 C:\Users\slab\AppData\Local\Temp\ipykernel_5232\2648232111.py: 86


[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.170515 [GHz]
EC 273.566712 [MHz]
EJ 13.616300 [GHz]
alpha -314.329034 [MHz]
dispersion 11.192453 [KHz]
Lq 11.995161 [nH]
Cq 70.806232 [fF]
T1 23.926334 [us]

**Coupling Properties**

tCqbus1 11.685186 [fF]
gbus1_in_MHz 151.783689 [MHz]
χ_bus1 -6.736454 [MHz]
1/T1bus1 6651.873399 [Hz]
T1bus1 23.926334 [us]
Bus-Bus Couplings


In [ ]:
default_options

{'cut_l': '300um+500um+15um+75um',
 'cut_h': '1200um',
 'gap': '75um',
 'w': '90um',
 'l': '600um',
 'r': '60um',
 'cpw_l': '50um',
 'coupling_gap': '15um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '300um',
 'coupling_pad_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': '10um',
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False'}

In [ ]:
df

In [ ]:
c1.sim.close()

In [ ]:
eig_all, renderer_hfss, hfss = dp.init_hfss_sim(nmode = 1,Ljs = [f'{qubit.L.to(u.nH).value:.2f}nH'])

In [ ]:
default_options

{'cut_l': '400um+500um+15um+25um',
 'cut_h': '1200um',
 'gap': '25um',
 'w': '90um',
 'l': '600um',
 'r': '60um',
 'cpw_l': '50um',
 'coupling_gap': '15um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '250um',
 'coupling_pad_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': '10um',
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'coupling_gd': '650um'}

In [294]:
qubit_q1 = TransmonQubit(4*u.GHz, ratio = 65)
default_options_q1 = {'cut_l': '350um+500um+50um',
 'cut_h': '1200um',
 'gap': '50um',
 'w': '90um',
 'l': '900um',
 'r': '60um',
 'cpw_l': '50um',
 'coupling_gap': '5um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '350um',
 'coupling_pad_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': design.variables['trace_width'],
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'layer' : '5'}
qubit_q2 = TransmonQubit(3.2*u.GHz, ratio = 65)
default_options_q2 = {'cut_l': '650um+200um+15um+12um',
 'cut_h': '1250um',
 'gap': '12um',
 'w': '90um',
 'l': '1100um',
 'r': '60um',
 'cpw_l': '50um',
 'coupling_gap': '15um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '400um',
 'coupling_pad_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': design.variables['trace_width'],
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'coupling_gd': '650um',
 'layer' : '5'}
qubit_q3 = TransmonQubit(4.8*u.GHz, ratio = 65)
default_options_q3 = {'cut_l': '250um+500um+15um+35um',
 'cut_h': '700um',
 'gap': '15um',
 'w': '80um',
 'l': '500um',
 'r': '60um',
 'cpw_l': '50um',
 'coupling_gap': '15um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '250um',
 'coupling_pad_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': design.variables['trace_width'],
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'layer' : '5'}

Optimal inductance L: 13.676453476080171 nH
Optimal inductance L: 17.095566845100272 nH
Optimal inductance L: 11.397044563400145 nH


### HFSS

In [ ]:
design.delete_all_components()

In [ ]:
default_options['junction'] = 'False'
q2  = DiffTransmonRounded(design,'Q2',options = default_options_q2)
gui.rebuild()

False
junction added
False
junction added


In [ ]:
eig_all.sim.close()

Warning! 12 COM references still alive
Ansys will likely refuse to shut down


In [ ]:
c1.sim.close()

In [ ]:
# eig_all.sim.setup.vars['Lj'] = f'{qubit.L.to(u.nH).value:.2f}nH'
dp.change_inductance([f'{(qubit_q2.L.to(u.nH).value):.2f}nH'],eig_all=eig_all,c1 = c1)
eig_all.sim.setup.vars['n_modes'] = 1
eig_all.sim.setup.n_modes = 1

In [ ]:
eig_all.sim.run( components = ['Q1'], open_terminations = [])

INFO 09:28PM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:28PM [load_ansys_project]: 	Opened Ansys App
INFO 09:28PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 09:28PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/slab/Documents/Ansoft/
	Project:   Project48
INFO 09:28PM [connect_design]: 	Opened active design
	Design:    Design_hfss [Solution type: Eigenmode]
INFO 09:28PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:28PM [connect]: 	Connected to project "Project48" and design "Design_hfss" 😀 

INFO 09:28PM [connect_design]: 	Opened active design
	Design:    Design_hfss [Solution type: Eigenmode]
09:28PM 07s WARNING [get_unique_component_ids]: The component=Q1 in highlight_qcomponents not in QDesign.
09:28PM 07s WARNING [render_design]: Unable to proceed with rendering. Please check selection.
INFO 09:28PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:28PM [analyze]: 

com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024349), None)

In [ ]:
q = 'q2'

In [ ]:
date = get_current_time_string()
hfss_result = (eig_all.sim.convergence_f).dropna()
hfss_result.to_csv('data/'+date+f'hfss_{q}.csv')

In [ ]:
eig_all.sim.convergence_f

,re(Mode(1)) [g]
Pass [],
1,2.233793
2,3.350035
3,3.703096
4,3.821440
5,3.895138
6,3.939699
7,3.973830
8,4.000294
9,4.019230


## Tee

In [122]:
design.delete_all_components()

In [123]:
res.pin

<Quantity 15. um>

In [124]:
TQ_options = dict(coupling_length='200 um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '2um',
               fillet = '30um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               layer = 5)
tee1 = CoupledLineTee(design,'tee1',options = TQ_options)
gui.rebuild()

In [69]:
# c1.sim.run(components = ['tee1'],open_terminations = [('tee1','prime_start'),('tee1','prime_end'),('tee1','second_end')])


INFO 09:44PM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:44PM [load_ansys_project]: 	Opened Ansys App
INFO 09:44PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 09:44PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/slab/Documents/Ansoft/
	Project:   Project48
INFO 09:44PM [connect_design]: 	Opened active design
	Design:    Design_hfss [Solution type: Eigenmode]
INFO 09:44PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:44PM [connect]: 	Connected to project "Project48" and design "Design_hfss" 😀 

INFO 09:44PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 09:44PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 09:44PM [analyze]: Analyzing setup Setup
INFO 09:45PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpvipc8_cy.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwe

In [70]:
# c1.sim.capacitance_matrix

In [ ]:
# qubit1 = TransmonQubit(4*u.GHz, ratio = 65)
# res1 = ReadoutResonator(6*u.GHz, short_on_one_end = False)
# dr1 = DispersiveReadout(qubit1,res1,5*u.ms)

Optimal inductance L: 13.676453476080171 nH


In [215]:
design.delete_all_components()

# Write design

In [ ]:
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from LaunchpadWirebondCustom import LaunchpadWirebondCustom


from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital


from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

## Wirebond Pads

In [256]:
Wirebond_opt = Dict(trace_width=design.variables['trace_width'],
                           trace_gap=design.variables['trace_gap'],
                           lead_length='240.5um',
                           pad_width='300um',
                           pad_height='500um',
                           pad_gap='100um',
                           taper_height='100um',
                           layer = str(qubit_layer),
                           pin_space = '200um')

Wirebond_opt['pos_x'] = '-2650um'
Wirebond_opt['pos_y'] = '2500um'
Wirebond_opt['orientation'] = '270'

wb_left = LaunchpadWirebondCustom(design, 'wb_left', options = Wirebond_opt)
Wirebond_opt['pos_x'] = '2650um'
wb_right = LaunchpadWirebondCustom(design, 'wb_right', options = Wirebond_opt)
gui.rebuild()

False
junction added
False
junction added
False
junction added


In [ ]:

# launch_options_l = dict(pos_x='-2650um', 
#                       pos_y='2812um', 
#                       orientation='270', 
#                       lead_length='70um', 
#                       trace_width = design.variables['trace_width'], 
#                       trace_gap = design.variables['cpw_gap'], 
#                       pos_x_end = '-2500um', 
#                       pos_y_end = '2500um',
#                       layer = 5)
# # Wirebond_opt['x_pos'] = '-2650um'
# # Wirebond_opt['y_pos'] = '2812um'
# # Wirebond_opt['orientation'] = '270'
# lp_left = LaunchpadWirebondCoupled(design, 'LP1', options = launch_options_l)
# gui.rebuild()

False
junction added
False
junction added
False
junction added


In [ ]:
# launch_options_r = dict(pos_x='2650um', 
#                       pos_y='2812um', 
#                       orientation='270', 
#                       lead_length='70um', 
#                       trace_width = design.variables['trace_width'], 
#                       trace_gap = design.variables['cpw_gap'], 
#                       pos_x_end = '2500um', 
#                       pos_y_end = '2500um',
#                       layer = 5)
# lp_right = LaunchpadWirebondCoupled(design, 'LP2', options = launch_options_r)
# gui.rebuild()

False
junction added
False
junction added
False
junction added


## Tee

In [221]:
TQ_options = dict(coupling_length='200 um + 90um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '2um',
               fillet = '90um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               pos_x = '-2000um',
               pos_y = '2000um', 
               mirror = True,
               layer = 5)
tee1 = CoupledLineTee(design,'tee1',options = TQ_options)
gui.rebuild()

In [222]:
TQ_options = dict(coupling_length='200 um + 90um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '2um',
               fillet = '90um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               pos_x = '2250um',
               pos_y = '2000um', 
               mirror = False,
               layer = 5)
tee2 = CoupledLineTee(design,'tee2',options = TQ_options)
gui.rebuild()

In [223]:
TQ_options = dict(coupling_length='200 um + 90 um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '2um',
               fillet = '90um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               pos_x = '1400um',
               pos_y = '2000um', 
               mirror = True,
               layer= 5)
tee3 = CoupledLineTee(design,'tee3',options = TQ_options)
gui.rebuild()

## Qubit

In [295]:
default_options_q1['pos_x'] = '-500um'
default_options_q1['pos_y'] = '1000um'
q1 = DiffTransmonRounded(design,'Q1',options = default_options_q1)
gui.rebuild()

False
junction added
False
junction added
False
junction added
False
junction added


In [296]:
default_options_q2['pos_x'] = '-1000um'
default_options_q2['pos_y'] = '-2000um'
default_options_q2['orientation'] = '180'
q2 = DiffTransmonRounded(design,'Q2',options = default_options_q2)
gui.rebuild()

False
junction added
False
junction added
False
junction added
False
junction added


In [297]:
default_options_q3['pos_x'] = '-900um'
default_options_q3['pos_y'] = '-700um'
default_options_q3['orientation'] = '180'
q3 = DiffTransmonRounded(design,'Q3',options = default_options_q3)
gui.rebuild()

False
junction added
False
junction added
False
junction added
False
junction added


10:42AM 58s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QMainWindowLayout::tabPosition called with out-of-bounds value '0'

10:42AM 59s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QMainWindowLayout::tabPosition called with out-of-bounds value '0'

10:42AM 59s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QMainWindowLayout::tabPosition called with out-of-bounds value '0'



In [266]:
res1 = ReadoutResonator(6*u.GHz, short_on_one_end = False)

In [267]:
route_options = dict(layer = 5,hfss_wire_bonds = False, 
                     fillet = '90um',
                     total_length = f'{res1.len}',
                     pin_inputs = dict(start_pin = dict(component = 'Q1', pin = 'cpw_stub'), 
                                                                end_pin = dict(component = 'tee1', pin = 'second_end')))
route_options['pin_width'] = design.variables['trace_width']
route_options['gap_width'] = design.variables['trace_gap']
route_options['trace_width'] = design.variables['trace_width']
route_options['trace_gap'] = design.variables['trace_gap']
route_options['lead'] = dict(start_straight = '600 um', end_straight = '400 um')
res1 = RouteMeander(design,name = 'res1',options = route_options)
gui.rebuild()

False
junction added
False
junction added
False
junction added


In [274]:
res2 = ReadoutResonator(5.5*u.GHz, short_on_one_end = False)

In [275]:
route_options = dict(layer = 5,hfss_wire_bonds = False, 
                     fillet = '80um',
                     total_length = f'{res2.len}',
                     pin_inputs = dict(start_pin = dict(component = 'Q2', pin = 'cpw_stub'), 
                                                                end_pin = dict(component = 'tee2', pin = 'second_end')))
route_options['pin_width'] = design.variables['trace_width']
route_options['gap_width'] = design.variables['trace_gap']
route_options['trace_width'] = design.variables['trace_width']
route_options['trace_gap'] = design.variables['trace_gap']
route_options['lead'] = dict(start_straight = '3000 um', end_straight = '400 um')
res2 = RouteMeander(design,name = 'res2',options = route_options)
gui.rebuild()

False
junction added
False
junction added
False
junction added


In [284]:
res3 = ReadoutResonator(6.5*u.GHz, short_on_one_end = False)

In [285]:
route_options = dict(layer = 5,hfss_wire_bonds = False, 
                     fillet = '90um',
                     total_length = f'{res3.len}',
                     pin_inputs = dict(start_pin = dict(component = 'Q3', pin = 'cpw_stub'), 
                                                                end_pin = dict(component = 'tee3', pin = 'second_end')))
route_options['pin_width'] = design.variables['trace_width']
route_options['gap_width'] = design.variables['trace_gap']
route_options['trace_width'] = design.variables['trace_width']
route_options['trace_gap'] = design.variables['trace_gap']
route_options['lead'] = dict(start_straight = '2300 um', end_straight = '400 um')
res3 = RouteMeander(design,name = 'res3',options = route_options)
gui.rebuild()

False
junction added
False
junction added
False
junction added


In [233]:
trans_ops = dp.trans_options

In [234]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'wb_left', pin = 'tie'),end_pin = dict(component = 'tee1', pin = 'prime_start'))
trans_ops['fillet'] = '100um'
trans1 = RouteMixed(design,name = 'trans1',options = trans_ops)
gui.rebuild()

False
junction added
False
junction added
False
junction added


In [235]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'tee3', pin = 'prime_start'),end_pin = dict(component = 'tee1', pin = 'prime_end'))
trans2 = RouteStraight(design,name = 'trans2',options = trans_ops)
gui.rebuild()

False
junction added
False
junction added
False
junction added


In [236]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'tee3', pin = 'prime_end'),end_pin = dict(component = 'tee2', pin = 'prime_start'))
trans3 = RouteStraight(design,name = 'trans3',options = trans_ops)
gui.rebuild()

False
junction added
False
junction added
False
junction added


In [237]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'wb_right', pin = 'tie'),end_pin = dict(component = 'tee2', pin = 'prime_end'))
trans_ops['fillet'] = '100um'
trans4 = RouteMixed(design,name = 'trans4',options = trans_ops)
gui.rebuild()

False
junction added
False
junction added
False
junction added


10:19AM 26s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QMainWindowLayout::tabPosition called with out-of-bounds value '0'



# GDS

In [298]:
# design.chips.main.size['size_x'] = '10mm'
# design.chips.main.size['size_y'] = '10mm'
# gui.rebuild()
a_gds = design.renderers.gds

In [299]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

In [300]:
a_gds.options['max_points'] = 3000
a_gds.options['fabricate'] = False

In [301]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
a_gds.options['cheese']['view_in_file']['main'][1] = False
a_gds.options['no_cheese']['view_in_file']['main'][1] = False

In [302]:
# a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['cheese']['view_in_file']['main']['1'] = True
# a_gds.options['no_cheese']['view_in_file']['main']['1'] = True



# a_gds.options['fabricate'] = False
a_gds.options.tolerance = '0.0000005'
a_gds.options.precision = '0.0000000005'

In [303]:
a_gds.options['cheese']

{'datatype': '100',
 'shape': '0',
 'cheese_0_x': '25um',
 'cheese_0_y': '25um',
 'cheese_1_radius': '50um',
 'view_in_file': {'main': {1: False,
   5: True,
   20: False,
   31: False,
   30: False}},
 'delta_x': '100um',
 'delta_y': '100um',
 'edge_nocheese': '150um'}

In [304]:
a_gds.options.cheese.edge_nocheese = '150um'
a_gds.options.no_cheese.buffer = '200um'
a_gds.options.cheese.cheese_1_radius = '50um'

In [305]:
# gui.rebuild()
a_gds.export_to_gds('qubit_design.gds')

10:43AM 15s WARNING [_import_junction_gds_file]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"c:\Users\slab\Desktop\Wendy-qiskit-code\resources".


1

In [172]:
a_gds.options['check_short_segments_by_scaling_fillet'] = '1.0'